In [13]:
import numpy as np
import pandas as pd
import os, time, warnings, random, requests, datetime, pytz, joblib
import functools as ft
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score, mean_squared_error
from google.cloud import storage
from io import BytesIO
import pandas_gbq

In [14]:
# !pip install pandas_gbq

In [15]:
time0 = time.time()
pull_time = datetime.datetime.now()
pull_time = pull_time.astimezone(pytz.timezone('America/New_York'))
pull_time = pull_time.replace(tzinfo=None)
now_time = (str(pull_time.month) + '_' + 
str(pull_time.day) + '_' +
str(pull_time.hour) + ':'  +
str(pull_time.minute) + ':' +
str(pull_time.second))

In [16]:
time1 = time.time()
# gsutil works on Vertex, but not in a Cloud Function...
data_bucket_name = 'pmykola-streaming-projects'
storage_client = storage.Client()
bucket = storage_client.get_bucket(data_bucket_name)
blobs_all = list(bucket.list_blobs())
blobs_specific = list(bucket.list_blobs(prefix='spg-stocks/data'))
temp_fnames = [blob.name for blob in blobs_specific]
datafiles = ['gs://' + data_bucket_name + '/' + fname for fname in temp_fnames]

# datafiles = !gsutil ls gs://pmykola-streaming-projects/spg-stocks/data
datafiles = [x for x in datafiles if ('auto_data_last_' in x) & ('pull_time' in x)]
f'time: {time.time()-time1} sec'

'time: 1.5147738456726074 sec'

In [17]:
time2 = time.time()
tempdf = pd.DataFrame(datafiles, columns=['filename'])
tempdf['year'] = 0
tempdf['month'] = 0
tempdf['day'] = 0
# display(tempdf.head())
# display(tempdf.iloc[0,:])
for i in range(tempdf.shape[0]):
    tempdf.loc[i,'year'] = tempdf.loc[i,'filename'].split("auto_data_last_",1)[1][:4]
    tail = tempdf.loc[i,'filename'].split("pull_time_",1)[1]
    tempdf.loc[i,'month'] = tail.split("_",1)[0]
    tempdf.loc[i,'day'] = (tail.split("_",1)[1]).split("_",1)[0]
    
# display(tempdf.head())

tempdf['date'] = pd.to_datetime(tempdf[['year', 'month', 'day']])
tempdf.sort_values(by='date', inplace=True)
# tempdf = tempdf.tail(4)
dates = tempdf.date.copy()
dates = [x for x in dates if x > datetime.date(2022, 12, 23)]
tempdf0 = tempdf.copy()
print(f'time: {time.time()-time2} sec')

time: 0.03754162788391113 sec


In [18]:
print(dates)
display(tempdf)

[Timestamp('2022-12-27 00:00:00'), Timestamp('2022-12-28 00:00:00'), Timestamp('2022-12-29 00:00:00'), Timestamp('2022-12-30 00:00:00'), Timestamp('2023-01-03 00:00:00'), Timestamp('2023-01-04 00:00:00'), Timestamp('2023-01-05 00:00:00'), Timestamp('2023-01-06 00:00:00'), Timestamp('2023-01-09 00:00:00'), Timestamp('2023-01-10 00:00:00'), Timestamp('2023-01-11 00:00:00'), Timestamp('2023-01-12 00:00:00'), Timestamp('2023-01-13 00:00:00'), Timestamp('2023-01-17 00:00:00'), Timestamp('2023-01-18 00:00:00'), Timestamp('2023-01-19 00:00:00'), Timestamp('2023-01-20 00:00:00'), Timestamp('2023-01-23 00:00:00'), Timestamp('2023-01-24 00:00:00'), Timestamp('2023-01-25 00:00:00'), Timestamp('2023-01-26 00:00:00')]


,filename,year,month,day,date
14,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,6,2022-12-06
15,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,7,2022-12-07
16,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,8,2022-12-08
17,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,9,2022-12-09
0,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,12,2022-12-12
1,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,13,2022-12-13
2,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,14,2022-12-14
3,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,15,2022-12-15
4,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,16,2022-12-16
5,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,19,2022-12-19


In [19]:
# this part implements for loop to do perfeval for each date in range
print(dates)

for last_day in dates:
    
    print(f'Last day is {last_day}')
    tempdf = tempdf0.copy()
    
    # last_day = dates[1]

    mask = [x <= last_day for x in list(tempdf.date)]
    tempdf = tempdf[mask]
    tempdf = tempdf.tail(4)
    display(tempdf)

    datafiles = list(tempdf.filename)
    print(datafiles)
    df = pd.read_csv(datafiles[0])
    print(f'time: {time.time()-time2} sec')

    df_new = pd.DataFrame(columns = df.columns)
    for file in datafiles:
        temp_df = pd.read_csv(file)
        df_new = pd.concat([df_new, temp_df], axis=0)
    print(f'time: {time.time()-time2} sec')

    # remove duplicates
    df_new.reset_index(inplace=True, drop=True)
    df_new.drop_duplicates(inplace=True)
    df_new.Datetime = pd.to_datetime(df_new.Datetime)
    df_new.sort_values(by='Datetime')
    print(f'time: {time.time()-time2} sec')

    df = df_new
    df.Datetime = pd.to_datetime(df.Datetime)
    df.drop_duplicates(inplace=True)
    df.sort_values(by='Datetime')

    df['time'] = df.Datetime.dt.time
    df['date'] = df.Datetime.dt.date

    df = df.fillna(method='ffill')
    dayclose = df[df.time==datetime.time(15, 58, 0)]
    dayopen = df[df.time==datetime.time(9, 30, 0)]
    dayopen.reset_index(drop=True, inplace=True)
    dayclose.reset_index(drop=True, inplace=True)
    dayclose.sort_values(by='date')
    print(f'time: {time.time()-time2} sec')

    ### now i wanna do feature engineering for all assets 

    asset_list = ['Spx', 'Nasdaq', 'Russel', 'EMXC', 'EEMA', 'EEM', 'VTHR']

    for asset in asset_list:

        df[asset + '_ret'] = 100*(df[asset]/df[asset].shift(1)-1)
        df['s_' + asset + '_ret_1prd'] = (100*(df[asset]/df[asset].shift(1)-1)).shift(1)
        df['s_' + asset + '_ret_2prd'] = (100*(df[asset]/df[asset].shift(2)-1)).shift(1)
        df['s_' + asset + '_ret_4prd'] = (100*(df[asset]/df[asset].shift(4)-1)).shift(1)
        # display(df.shape, df.head(5))

        df.loc[df.time < datetime.time(9, 32, 0), 's_' + asset + '_1prd'] = np.nan
        df.loc[df.time < datetime.time(9, 33, 0), 's_' + asset + '_2prd'] = np.nan
        df.loc[df.time < datetime.time(9, 35, 0), 's_' + asset + '_4prd'] = np.nan

        dayopen.rename(columns={asset:asset+'_open'}, inplace=True)
        # dayopen.head()
        dayclose.rename(columns={asset:asset+'_close'}, inplace=True)
        dayclose_l1 = dayclose.copy()
        dayclose_l2 = dayclose.copy()
        dayclose_l1[asset+'_close_l1'] = dayclose_l1[asset+'_close'].shift(1)
        dayclose_l2[asset+'_close_l2'] = dayclose_l2[asset+'_close'].shift(2)

        df = pd.merge(df, dayopen[['date', asset + '_open']], on=['date'], how='left')
        df = pd.merge(df, dayclose_l1[['date', asset + '_close_l1']], on=['date'], how='left')
        df = pd.merge(df, dayclose_l2[['date', asset + '_close_l2']], on=['date'], how='left')

        df['s_' + asset + '_ret_open'] = (100*(df[asset]/df[asset + '_open']-1)).shift(1)
        df['s_' + asset + '_ret_close1'] = (100*(df[asset]/df[asset + '_close_l1']-1)).shift(1)
        df['s_' + asset + '_ret_close2'] = (100*(df[asset]/df[asset + '_close_l2']-1)).shift(1)

        cols_todrop = [x for x in list(df.columns) if asset in x and 'ret' not in x]
        df.drop(columns = cols_todrop, inplace=True)
    print(f'time: {time.time()-time2} sec')

    ### do prediction ###

    storage_client = storage.Client()
    bucket_name='pmykola-streaming-projects'
    model_path='spg-stocks/artifacts/en_model.pkl'

    bucket = storage_client.get_bucket(bucket_name)
    blob = bucket.blob(model_path)
    model_file = BytesIO()
    blob.download_to_file(model_file)
    trained_model=joblib.load(model_file)

    # df.date.max()
    this_day = df.loc[df.date == df.date.max()]
    print(f'{this_day.shape[0]} observations this day')
    X = this_day.copy()
    X.drop(columns = ['Datetime',
                  'time', 
                  'date', 
                  'Spx_ret', 
                  'Nasdaq_ret', 
                  'Russel_ret', 
                  'EEMA_ret', 
                  'EEM_ret', 
                  'EMXC_ret', 
                  'VXUS_ret', 
                  'VTHR_ret'], 
                  inplace=True,
                  errors = 'ignore')

    if(X.count().sum() < X.shape[1]):
        print(f'''There are {X.shape[1] - X.count().sum()} missing values. 
              There will be an error''')

    y = this_day.VTHR_ret
    y_hat = trained_model.predict(X)

    model_rmse = mean_squared_error(y, y_hat)
    constant_rmse = mean_squared_error(y, np.zeros(len(y)))

    performance = pd.DataFrame([[100*(r2_score(y, y_hat)), model_rmse, constant_rmse, 100*(1-model_rmse/constant_rmse)]], 
                          columns = ['R2', 'model_rmse', 'constant_rmse', 'rmse_improvement'])
    performance['date'] = df.date.max()
    print(f'time: {time.time()-time2} sec')

    file_name = 'spg-stocks/artifacts/performance-data/' + \
    'm1_performance_' + \
    str(df.date.max().year) + \
    str(df.date.max().month) + \
    str(df.date.max().day) + \
    '_pull_time_' + \
    now_time + \
    '.csv'

    storage_client = storage.Client()
    bucket_name = 'pmykola-streaming-projects'
    BUCKET = storage_client.get_bucket(bucket_name)

    blob = BUCKET.blob(file_name)
    blob.upload_from_string(performance.to_csv(), 'text/csv')
    print('Upload to Cloud Storage complete.')

    project_id = 'valid-heuristic-369117'
    bucket_path = 'gs://pmykola-streaming-projects/spg-stocks/artifacts/performance-data/'
    table_id = 'spg_stocks.daily_performance'

    performance.rename(columns={'date':'ddate'}, inplace=True)
    pandas_gbq.to_gbq(performance, table_id, project_id=project_id, if_exists='append')

    result = ('Success: ' + file_name + ' upload complete. ' + 
    'Total time: ' + str(time.time()-time0)[:6] + 'sec')
    print('response', result)
    print(f'time: {time.time()-time2} sec')
    
    

[Timestamp('2022-12-27 00:00:00'), Timestamp('2022-12-28 00:00:00'), Timestamp('2022-12-29 00:00:00'), Timestamp('2022-12-30 00:00:00'), Timestamp('2023-01-03 00:00:00'), Timestamp('2023-01-04 00:00:00'), Timestamp('2023-01-05 00:00:00'), Timestamp('2023-01-06 00:00:00'), Timestamp('2023-01-09 00:00:00'), Timestamp('2023-01-10 00:00:00'), Timestamp('2023-01-11 00:00:00'), Timestamp('2023-01-12 00:00:00'), Timestamp('2023-01-13 00:00:00'), Timestamp('2023-01-17 00:00:00'), Timestamp('2023-01-18 00:00:00'), Timestamp('2023-01-19 00:00:00'), Timestamp('2023-01-20 00:00:00'), Timestamp('2023-01-23 00:00:00'), Timestamp('2023-01-24 00:00:00'), Timestamp('2023-01-25 00:00:00'), Timestamp('2023-01-26 00:00:00')]
Last day is 2022-12-27 00:00:00


,filename,year,month,day,date
7,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,21,2022-12-21
8,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,22,2022-12-22
9,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,23,2022-12-23
10,gs://pmykola-streaming-projects/spg-stocks/dat...,2022,12,27,2022-12-27


['gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_20221221_pull_time_12_21_16:10:4.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_20221222_pull_time_12_22_16:10:5.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_20221223_pull_time_12_23_16:10:5.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_20221227_pull_time_12_27_16:10:8.csv']
time: 1.190131425857544 sec
time: 4.944441318511963 sec
time: 4.948533535003662 sec
time: 4.955952167510986 sec
time: 5.10336971282959 sec


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


196 observations this day
time: 6.265880823135376 sec
Upload to Cloud Storage complete.


...

['gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_2023123_pull_time_1_23_16:10:6.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_2023124_pull_time_1_24_16:10:6.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_2023125_pull_time_1_25_16:10:6.csv', 'gs://pmykola-streaming-projects/spg-stocks/data/auto_data_last_2023126_pull_time_1_26_16:10:5.csv']
time: 195.56451845169067 sec
time: 197.28751969337463 sec
time: 197.29165244102478 sec
time: 197.29930067062378 sec
time: 197.4456832408905 sec


/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


196 observations this day
time: 197.71008849143982 sec
Upload to Cloud Storage complete.


100%|██████████| 1/1 [00:00<00:00, 9039.45it/s]

response Success: spg-stocks/artifacts/performance-data/m1_performance_2023126_pull_time_1_27_10:4:13.csv upload complete. Total time: 205.04sec
time: 203.49264121055603 sec
